In [1]:
### WLP_Salmon_Spawning_Survey_DataJoinSummary_v1.py
### Version: 3/7/2022
### Author: Khem So, khem_so@fws.gov, (503) 231-6839
### Abstract: This Python 3 script pulls data from the HI Waterbirds Reproductive Success ArcGIS Online feature service and performs joins and merges to result in a combined CSV dataset.

In [2]:
import arcpy
import pandas as pd
from arcgis import GIS
import time, os, fnmatch, shutil
import openpyxl

In [3]:
### ArcGIS Online stores date-time information in UTC by default. This function uses the pytz package to convert time zones and can be used to convert from UTC ("UTC") to localized time. For example, localized "US/Pacific" is either Pacific Standard Time UTC-8 or Pacific Daylight Time UTC-7 depending upon time of year.
from datetime import datetime
from pytz import timezone
def change_timezone_of_field(df, source_date_time_field, new_date_time_field_suffix, source_timezone, new_timezone):
    """Returns the values in *source_date_time_field* with its timezone converted to a new timezone within a new field *new_date_time_field*
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field_suffix: Suffix appended to the end of the name of the source datetime field. This is used to create the new date time field name.
    : param source_timezone: The name of the source timezone
    : param new_timezone: The name of the converted timezone. For possible values, see https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568
    """
    # Define the source timezone in the source_date_time_field
    df[source_date_time_field] = df[source_date_time_field].dt.tz_localize(source_timezone)
    # Define the name of the new date time field
    new_date_time_field = source_date_time_field + new_date_time_field_suffix
    # Convert the datetime in the source_date_time_field to the new timezone in a new field called new_date_time_field
    df[new_date_time_field] = df[source_date_time_field].dt.tz_convert(new_timezone)

In [4]:
### This function converts Python datetime64 fields to %m/%d/%Y %H:%M:%S %Z%z format
def archive_dt_field(df):
    """Selects fields with data types of 'datetime64[ns, UTC]','datetime64[ns, US/Pacific]' and converts to %m/%d/%Y %H:%M:%S %Z%z format for archiving to Excel
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    """
    archive_dt_field_list = df.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]'])
    for col in archive_dt_field_list:
        df[col] = df[col].dt.strftime('%m/%d/%Y %H:%M:%S %Z%z')

In [5]:
### Allow authentication via login to U.S. Fish & Wildlife Service ArcGIS Online account via ArcGIS Pro
gis = GIS("pro")

In [6]:
### Enter year of interest
# uncomment next line to use ArcGIS interface, otherwise hard coding year
# year = arcpy.GetParameterAsText(0)
year = "2021"

In [7]:
### Enter path for local file saving
# uncomment next line to use ArcGIS interface, otherwise hard coding out_workspace
# out_workspace = arcpy.GetParameterAsText(1)
out_workspace = "C:/Users/kso/Desktop/"

In [8]:
### Create timestamp for file naming
t = time.localtime()
timestamp = time.strftime('%Y-%m-%d_%H%M', t)

In [9]:
### Paths to ArcGIS Online data
# To populate Service ItemId, go to Feature Service webpage and in bottom right corner, click on the View link.
# Current Feature Service webpage: https://fws.maps.arcgis.com/home/item.html?id=758626eec0fc4bc1a72b4e4c9bd1023c
ServiceItemID = gis.content.get("758626eec0fc4bc1a72b4e4c9bd1023c")

### There are separate methods for pulling spatial versus non-spatial data into Python. Spatial layers will become Spatially Enabled DataFrame objects. Non-spatial data will become regular pandas DataFrame objects.
## Define variables pointing to spatial layers
MetadataLyr = ServiceItemID.layers[0]
LiveFishLyr = ServiceItemID.layers[1]
CarcassLyr = ServiceItemID.layers[2]
## Create Spatially Enabled DataFrame objects
sedfMetadata = pd.DataFrame.spatial.from_layer(MetadataLyr)
sedfLiveFishLocation = pd.DataFrame.spatial.from_layer(LiveFishLyr)
sedfCarcassLocation = pd.DataFrame.spatial.from_layer(CarcassLyr)

## Define variables point to non-spatial (tabular) data
Observer = r"https://services.arcgis.com/QVENGdaPbd4LUkLV/arcgis/rest/services/service_c555c76424ca452d8dab8de4f8c25000/FeatureServer/3"

## Convert AGOL table to NumPy Array and then to pandas DataFrames
naObserver = arcpy.da.TableToNumPyArray(Observer,["objectid","globalid","strFirstName","strLastName","parentglobalid","CreationDate","Creator","EditDate","Editor"])
dfObserver = pd.DataFrame(naObserver)

In [10]:
### Use change_timezone_of_field function to convert all datetime fields in dataframe from UTC to Pacific within new field with _Pacific suffix
for col in sedfMetadata.columns:
     if sedfMetadata[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(sedfMetadata, col, "_Pacific", "UTC", "US/Pacific")

for col in sedfLiveFishLocation.columns:
     if sedfLiveFishLocation[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(sedfLiveFishLocation, col, "_Pacific", "UTC", "US/Pacific")

for col in sedfCarcassLocation.columns:
     if sedfCarcassLocation[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(sedfCarcassLocation, col, "_Pacific", "UTC", "US/Pacific")

for col in dfObserver.columns:
     if dfObserver[col].dtype == 'datetime64[ns]':
         change_timezone_of_field(dfObserver, col, "_Pacific", "UTC", "US/Pacific")

In [11]:
### Filter sedfMetadata by single year
sedfMetadataYYYY = sedfMetadata[sedfMetadata["dtmDate"].dt.strftime('%Y') == year]

In [12]:
### Export raw data frames as backup
## Use archive_dt_field function to convert Python date time into format Excel can read more easily
archive_dt_field(sedfMetadata)
archive_dt_field(sedfLiveFishLocation)
archive_dt_field(sedfCarcassLocation)
archive_dt_field(dfObserver)

## Create export paths for backup and writes to Excel spreadsheet
writer = pd.ExcelWriter(os.path.join(out_workspace,('WLP_Salmon_Spawning_Survey_BKUP_' + timestamp + '.xlsx')))
sedfMetadata.to_excel(writer, 'Metadata')
sedfLiveFishLocation.to_excel(writer, 'Live Fish')
sedfCarcassLocation.to_excel(writer, 'Carcasses')
dfObserver.to_excel(writer, 'Observers')
writer.save()

In [13]:
### Create dfObserver2 data frame with concatenated surveyor names grouped by parentglobalid
## Clean up names
dfObserver["strFirstName"] = dfObserver["strFirstName"].str.strip()
dfObserver["strLastName"] = dfObserver["strLastName"].str.strip()

## Process dfObserver to get single concatenated field for full name
dfObserver["strFullName"] = dfObserver["strFirstName"] + " " + dfObserver["strLastName"]

## Process dfObserver to remove curly brackets to allow for join based on GUID
dfObserver = dfObserver.replace("{","", regex=True)
dfObserver = dfObserver.replace("}","", regex=True)

## Process dfObserver to get concatenated list of full surveyor names by survey
dfObserver2 = dfObserver[["parentglobalid", "strFullName"]]
dfObserver2 = dfObserver2.groupby("parentglobalid").agg({"strFullName": ', '.join})

In [14]:
### Join sedfMetadataYYYY with dfObserver
dfMetadataObserver = pd.merge(sedfMetadataYYYY,dfObserver2, how="left", left_on="globalid", right_on="parentglobalid")

In [15]:
### Manipulate date/time fields in dfMetadataObserver
## Strip time from dtmDate_Pacific
dfMetadataObserver["dtmDate_Pacific"] = dfMetadataObserver["dtmDate_Pacific"].dt.strftime('%m/%d/%Y')

## Calculate total survey time
dfMetadataObserver["dtmManualTimeStart_dt"] = dfMetadataObserver["dtmDate_Pacific"] + " " + dfMetadataObserver["dtmManualTimeStart"]
dfMetadataObserver["dtmManualTimeStart_dt"] = pd.to_datetime(dfMetadataObserver["dtmManualTimeStart_dt"],format="%m/%d/%Y %H:%M")

dfMetadataObserver["dtmManualTimeEnd_dt"] = dfMetadataObserver["dtmDate_Pacific"] + " " + dfMetadataObserver["dtmManualTimeEnd"]
dfMetadataObserver["dtmManualTimeEnd_dt"] = pd.to_datetime(dfMetadataObserver["dtmManualTimeEnd_dt"],format="%m/%d/%Y %H:%M")

dfMetadataObserver["dtmManualTimeTotal"] = dfMetadataObserver["dtmManualTimeEnd_dt"] - dfMetadataObserver["dtmManualTimeStart_dt"]

dfMetadataObserver["dtmManualTimeTotal"] = (dfMetadataObserver["dtmManualTimeTotal"]).astype(str)

In [16]:
### Reset dfMetadataObserver in desired order and drop unneeded fields
dfMetadataObserver = dfMetadataObserver[["globalid", "strStream", "dtmDate_Pacific", "strFullName", "strTideStart", "strWeather", "dtmManualTimeStart", "dtmManualTimeTurn", "dtmManualTimeEnd", "dtmManualTimeTotal", "strStreamFlow", "strViewingConditions", "strViewingConditionsComments", "ysnLiveFish", "ysnCarcasses", "strComments", "CreationDate_Pacific"]]

In [17]:
### Join dfMetadataObserver with sedfLiveFishLocation
dfMetadataObserverLiveFish = pd.merge(dfMetadataObserver,sedfLiveFishLocation, how="inner", left_on="globalid", right_on="parentglobalid")

## Reset dfMetadataObserverLiveFish in desired order and drop unneeded fields
dfMetadataObserverLiveFish = dfMetadataObserverLiveFish[['globalid_x', 'strStream', 'dtmDate_Pacific', 'ysnLiveFish', 'globalid_y', 'strLiveSpecies', 'strLiveSex', 'ysnPairs', 'ysnReddBuilding', 'intNumRedds', 'strLiveFishRedd', 'strReddID', 'SHAPE', 'CreationDate_Pacific_x']]
## Define dfMetadataObserverLiveFish sort order
dfMetadataObserverLiveFish = dfMetadataObserverLiveFish.sort_values(by=["strStream", "dtmDate_Pacific"])

In [18]:
### Join dfMetadataObserver with sedfCarcassLocation
dfMetadataObserverCarcasses = pd.merge(dfMetadataObserver,sedfCarcassLocation, how="inner", left_on="globalid", right_on="parentglobalid")
## Reset dfMetadataObserverCarcasses in desired order and drop unneeded fields
dfMetadataObserverCarcasses = dfMetadataObserverCarcasses[['globalid_x', 'strStream', 'dtmDate_Pacific', 'ysnCarcasses', 'globalid_y', 'strCarcassSpecies', 'strCarcassSex', 'strDecomposedFresh', 'intNumCarcasses', 'ysnCountedLast', 'SHAPE', 'CreationDate_Pacific_x']]
## Define dfMetadataObserverCarcasses sort order
dfMetadataObserverCarcasses = dfMetadataObserverCarcasses.sort_values(by=["strStream", "dtmDate_Pacific"])

In [19]:
### Data entered prior to 11/5/2021 are in different format so before/after data frames needed
dfMetadataObserverLiveFish_before20211105 = dfMetadataObserverLiveFish[(dfMetadataObserverLiveFish['CreationDate_Pacific_x'] < "11/05/2021")]
dfMetadataObserverLiveFish_after20211105 = dfMetadataObserverLiveFish[(dfMetadataObserverLiveFish['CreationDate_Pacific_x'] >= "11/05/2021")]
dfMetadataObserverCarcasses_before20211105 = dfMetadataObserverCarcasses[(dfMetadataObserverCarcasses['CreationDate_Pacific_x'] < "11/05/2021")]
dfMetadataObserverCarcasses_after20211105 = dfMetadataObserverCarcasses[(dfMetadataObserverCarcasses['CreationDate_Pacific_x'] >= "11/05/2021")]
dfMetadataObserverLiveFish_before20211105

dfMetadataObserverLiveFish_before20211105 = dfMetadataObserverLiveFish_before20211105.copy()
dfMetadataObserverLiveFish_after20211105 = dfMetadataObserverLiveFish_after20211105.copy()
dfMetadataObserverCarcasses_before20211105 = dfMetadataObserverCarcasses_before20211105.copy()
dfMetadataObserverCarcasses_after20211105 = dfMetadataObserverCarcasses_after20211105.copy()

In [20]:
### Create fields for counting live fish entered before 11/5/2021
dfMetadataObserverLiveFish_before20211105.loc[dfMetadataObserverLiveFish_before20211105['ysnReddBuilding'] == "yes", ['intReddBuilding']] = 1
dfMetadataObserverLiveFish_before20211105.loc[dfMetadataObserverLiveFish_before20211105['ysnPairs'] == "yes", ['dblPairs']] = 0.5
dfMetadataObserverLiveFish_before20211105.loc[dfMetadataObserverLiveFish_before20211105['strLiveSex'] == "M", ['intMales']] = 1
dfMetadataObserverLiveFish_before20211105.loc[dfMetadataObserverLiveFish_before20211105['strLiveSex'] == "F", ['intFemales']] = 1
dfMetadataObserverLiveFish_before20211105.loc[dfMetadataObserverLiveFish_before20211105['strLiveSex'] == "Unk", ['intUnknown']] = 1

## Group by GUID, stream, date, and species; sum the numeric fields
dfLiveFishSummary1 = dfMetadataObserverLiveFish_before20211105.groupby(['globalid_x', 'strStream', 'dtmDate_Pacific', 'strLiveSpecies']).sum()

## Create field for sum of live fish
dfLiveFishSummary1['intLiveFish'] = dfLiveFishSummary1[['intMales', 'intFemales', 'intUnknown']].sum(axis=1)

In [22]:
### Create fields for counting live fish entered after 11/5/2021
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['ysnReddBuilding'] == "yes", ['intReddBuilding']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['ysnPairs'] == "yes", ['dblPairs']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['ysnPairs'] == "yes", ['intMales']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['ysnPairs'] == "yes", ['intFemales']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['strLiveSex'] == "M", ['intMales']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['strLiveSex'] == "F", ['intFemales']] = 1
dfMetadataObserverLiveFish_after20211105.loc[dfMetadataObserverLiveFish_after20211105['strLiveSex'] == "Unk", ['intUnknown']] = 1
dfMetadataObserverLiveFish_after20211105.loc[((dfMetadataObserverLiveFish_after20211105['strLiveFishRedd'] == "Live Fish and Redd") | (dfMetadataObserverLiveFish_after20211105['strLiveFishRedd'] == "Redd")), ['intNumRedds']] = 1

## Group by GUID, stream, date, and species; sum the numeric fields
dfLiveFishSummary2 = dfMetadataObserverLiveFish_after20211105.groupby(['globalid_x', 'strStream', 'dtmDate_Pacific', 'strLiveSpecies']).sum()

## Create field for sum of live fish
dfLiveFishSummary2['intLiveFish'] = dfLiveFishSummary2[['intMales', 'intFemales', 'intUnknown']].sum(axis=1)

In [29]:
### Combine live fish data from before and after 11/5/2021
dfLiveFishSummary = pd.concat([dfLiveFishSummary1, dfLiveFishSummary2])

In [30]:
### Copy dfMetadataObserver as start of dfSummary
dfSummary = dfMetadataObserver.copy()
# Calculate zeroes
dfSummary.loc[dfSummary['ysnLiveFish'] == "no", ['intLiveFish']] = 0
dfSummary.loc[dfSummary['ysnCarcasses'] == "no", ['intCarcasses']] = 0
# Join
dfSummary = pd.merge(dfSummary,dfLiveFishSummary, how="left", left_on="globalid", right_on="globalid_x")
dfSummary

,globalid,strStream,dtmDate_Pacific,strFullName,strTideStart,strWeather,dtmManualTimeStart,dtmManualTimeTurn,dtmManualTimeEnd,dtmManualTimeTotal,...,CreationDate_Pacific,intLiveFish_x,intCarcasses,intNumRedds,intReddBuilding,dblPairs,intMales,intFemales,intUnknown,intLiveFish_y
0,b4814934-9f33-44a1-bfa8-7ba7a0840fe9,Porter Point Stream,10/28/2021,"Kelsey Lotz, Jonathan Bates","Low, rising",Overcast/Drizzle,12:21,None,13:06,0 days 00:45:00,...,10/29/2021 08:13:31 PDT-0700,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cebcb048-63e3-463c-8ce8-882938ca5cea,Lost Creek,10/29/2021,Kelsey Lotz,"High, falling",Clear,09:49,10:15,10:30,0 days 00:41:00,...,10/29/2021 10:36:32 PDT-0700,NaN,0.0,1.0,5.0,2.0,8.0,17.0,5.0,30.0
2,2a9082d2-c328-4ff2-abbe-92f5e570beca,Chum Creek,10/29/2021,Kelsey Lotz,"High, falling",Clear,10:37,None,11:27,0 days 00:50:00,...,10/29/2021 11:23:39 PDT-0700,NaN,NaN,0.0,0.0,2.0,6.0,10.0,6.0,22.0
3,404fe8a4-90cd-4ba5-9036-4450a6fa0a8c,McCollum Stream,11/02/2021,Kelsey Lotz,"High, rising",Showers,10:37,None,11:20,0 days 00:43:00,...,11/02/2021 15:45:34 PDT-0700,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06aa11f4-c393-4a3d-877a-f534b20e2cb9,Lost Creek,11/05/2021,Kelsey Lotz,"Low, rising",Partly cloudy,09:17,None,11:50,0 days 02:33:00,...,11/08/2021 09:07:05 PST-0800,NaN,NaN,4.0,42.0,21.0,51.0,59.0,30.0,140.0
5,b18399a6-4756-4f9a-8700-3bd39bcf24a7,Chum Creek,11/05/2021,Kelsey Lotz,"High, rising",Rain,12:06,None,13:09,0 days 01:03:00,...,11/08/2021 09:07:06 PST-0800,NaN,NaN,0.0,10.0,9.0,25.0,20.0,9.0,54.0
6,8c146530-ef99-4a4f-beb5-e3f9d79bc2e6,Omeara Creek,11/08/2021,Kelsey Lotz,"Low, rising",Clear,09:07,None,10:34,0 days 01:27:00,...,11/08/2021 11:17:01 PST-0800,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,b19b7a07-f769-45b1-a140-2bd8e03879d5,Headquarters Creek,11/08/2021,Kelsey Lotz,"Half, rising",Clear,12:07,None,12:45,0 days 00:38:00,...,11/08/2021 14:07:03 PST-0800,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,c2bdab4e-13fd-48f1-ae93-cae3f8cf2654,Lewis Stream,11/08/2021,"Kelsey Lotz, William Ritchie",NA,None,15:03,15:21,15:45,0 days 00:42:00,...,11/10/2021 09:52:19 PST-0800,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,611223c4-b37a-4d76-854f-e3690b52bed0,Lost Creek,11/09/2021,Kelsey Lotz,NA,Partly cloudy,10:05,None,14:38,0 days 04:33:00,...,11/10/2021 09:52:19 PST-0800,NaN,NaN,26.0,19.0,45.0,81.0,74.0,14.0,169.0


In [28]:
### Export data frames
## Use archive_dt_field function to convert Python date time into format Excel can read more easily
archive_dt_field(dfMetadataObserver)
archive_dt_field(dfMetadataObserverLiveFish)
archive_dt_field(dfMetadataObserverCarcasses)
# delete later
archive_dt_field(dfMetadataObserverLiveFish_before20211105)
archive_dt_field(dfMetadataObserverLiveFish_after20211105)
archive_dt_field(dfMetadataObserverCarcasses_before20211105)
archive_dt_field(dfMetadataObserverCarcasses_after20211105)
archive_dt_field(dfLiveFishSummary1)
archive_dt_field(dfLiveFishSummary2)
archive_dt_field(dfSummary)
    
## Create export paths for backup and writes to Excel spreadsheet
writer = pd.ExcelWriter(os.path.join(out_workspace,('WLP_Salmon_Spawning_Survey_' + year + '_' + timestamp + '.xlsx')))
dfMetadataObserver.to_excel(writer, 'Metadata')
dfMetadataObserverLiveFish.to_excel(writer, 'Live Fish')
dfMetadataObserverCarcasses.to_excel(writer, 'Carcasses')
# delete later
dfMetadataObserverLiveFish_before20211105.to_excel(writer, 'LiveFish_before20211105')
dfMetadataObserverLiveFish_after20211105.to_excel(writer, 'LiveFish_after20211105')
dfMetadataObserverCarcasses_before20211105.to_excel(writer, 'Carcasses_before20211105')
dfMetadataObserverCarcasses_after20211105.to_excel(writer, 'Carcasses_after20211105')
dfLiveFishSummary1.to_excel(writer, 'LiveFishSummary1')
dfLiveFishSummary2.to_excel(writer, 'LiveFishSummary2')
dfSummary.to_excel(writer, 'Summary')
writer.save()